In [1]:
import pandas as pd
import numpy as np
import glob
import os

tests文件夹里面放每个样本的测试结果 e.g.,{顶棚.xlsx, 方向盘.xlsx}
- 每个文件第一行为表头：{组分，浓度}
- 第一列为组分名称，第二列为组分浓度
----
properties.xlsx包含每个组分的属性
- 第一行表头及属性名称
- 第一列为组分名称
- 2到最后列为属性值
---

In [2]:
path = './tests'                       #GCMS检测结果文件夹的路径
properties_file = './properties.xlsx'  #各组分属性值表的路径
result_file = './data.xls'             #结果保存路径

把tests文件夹里的所有样本对齐
- 对齐
- 补零

In [3]:
allFiles = glob.glob(path + "/*.xlsx")
parts = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_excel(file_,index_col=0, header=0)
    list_.append(df)
parts = pd.concat(list_,axis=1)
parts.columns = list(map(lambda x:x[len(path+'/'):-len('.xlsx')],allFiles))
parts = parts.fillna(0)#补零

In [4]:
parts

,把手,方向盘,方向盘1,方向盘2,方向盘3,顶棚,顶棚2
A,0.0,0.0,0.0,0.0,0.0,0.0,2.0
C,2.0,0.3,0.3,0.3,0.3,2.0,2.0
D,2.0,0.4,0.0,0.0,1.0,0.0,0.0
E,0.0,0.2,0.2,0.2,0.2,0.0,0.0
G,12.0,0.0,0.0,0.0,0.0,5.0,5.0
M,0.0,0.0,0.0,0.0,0.0,4.0,4.0
S,1.0,0.0,0.2,0.2,0.2,1.0,1.0
W,0.0,0.0,0.0,2.0,2.0,0.0,0.0


In [5]:
def scaler(x):
    #归一化
    return x/x.sum(axis=0) 

In [6]:
properties = pd.read_excel(properties_file,index_col=0)
#sproperties = properties.apply(scaler)#归一化
Pw = pd.Series([.9,.1,0],index=properties.columns,name='属性权重')#每个属性的权重
properties.shape

(26, 3)

properties共26个，查看前5个

In [17]:
properties

,毒性,分子量,XX
成分,,,
A,0.100,2.000000,2.0
B,0.010,3.000000,2.0
C,100.000,20.000000,13.0
D,1.000,26.333333,23.0
E,50.950,35.333333,28.5
F,61.219,44.333333,35.9
G,71.488,53.333333,43.3
H,81.757,62.333333,50.7
I,92.026,71.333333,58.1


In [8]:
Pw#每个属性的权重

毒性     0.9
分子量    0.1
XX     0.0
Name: 属性权重, dtype: float64

In [16]:
properties.loc[parts.index,:]

,毒性,分子量,XX
A,0.100,2.000000,2.0
C,100.000,20.000000,13.0
D,1.000,26.333333,23.0
E,50.950,35.333333,28.5
G,71.488,53.333333,43.3
M,133.102,107.333333,87.7
S,194.716,161.333333,132.1
W,235.792,197.333333,161.7


In [9]:
w = scaler(properties.dot(Pw)[parts.index])

In [10]:
parts_weighted = parts.T*w
parts_weighted

,A,C,D,E,G,M,S,W
把手,0.000000,0.239355,0.009193,0.000000,1.087595,0.00000,0.248952,0.00000
方向盘,0.000000,0.035903,0.001839,0.012849,0.000000,0.00000,0.000000,0.00000
方向盘1,0.000000,0.035903,0.000000,0.012849,0.000000,0.00000,0.049790,0.00000
方向盘2,0.000000,0.035903,0.000000,0.012849,0.000000,0.00000,0.049790,0.60345
方向盘3,0.000000,0.035903,0.004596,0.012849,0.000000,0.00000,0.049790,0.60345
顶棚,0.000000,0.239355,0.000000,0.000000,0.453165,0.67917,0.248952,0.00000
顶棚2,0.000754,0.239355,0.000000,0.000000,0.453165,0.67917,0.248952,0.00000


结果导出到data.xls

In [11]:
parts_weighted.to_excel(result_file)

# 之后建模：chempattern